###` Ce document Google collab a pour but de présenter le pipeline utilisé lors du projet BILL ainsi que les différentes commandes des différents outils qui le constituent `###

Document réalisé par :  
MECHKOURI Nazim : sabri-nazim.mechkouri@etu.umontpellier.fr
& OLLAGNIER Mathilde : mathilde.ollagnier@etu.umontpellier.fr

# Partie script personnalisé : VCFReader #

####pour toute information concernant le fonctionnement du script, son usage ou le code, veuillez vous dirigez vers le lien GitHub suivant : https://github.com/Senwei1/BILL####

# Informations supplémentaires #
__authors__ = ("Nazim MECHKOURI")

__contact__ = ("sabri-nazim.mechkouri@etu.umontpellier.fr")

__version__ = "1.0.2"

__date__ = "17/02/2023"

__licence__ =  

        "This program is free software: you can redistribute it and/or modify
        it under the terms of the GNU General Public License as published by
        the Free Software Foundation, either version 3 of the License, or
        (at your option) any later version.

        This program is distributed in the hope that it will be useful,
        but WITHOUT ANY WARRANTY; without even the implied warranty of
        MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
        GNU General Public License for more details.
        
        You should have received a copy of the GNU General Public License
        along with this program. If not, see <https://www.gnu.org/licenses/>."


In [ ]:
###      Configuration de l'environnement pour snakemake :      ####

#Tout d'abord, l'arborescence du repertoire doit ressembler à cette configuration : #

 Arborescence à respecter : 
 Analyse
 |____P30
 | |__P30.C6
 |     |__P30.C6_all.fastq.gz
 | |____P20
 | |__P20
 |     |__P20.C1_all.fastq.gz
 |____BILL
 | |___sequence.fasta
 | |___ Script
 |       |__cluster_config.yaml
 |       |__config.yaml
 |       |__extract_sv.py
 |       |__Snakefile
 |       |__snakemake.sh
 | 
 |________________________________________________



#ensuite, la configuration du snakemake pour lui accorder le chemin des fichiers d'entrée et de sortie : #
raw_dir: "analyse/"
ref_dir: "data_copie/"
res_dir: "analyse/"
log_dir: "LOGS/Ref"
ben_dir: "BENCHMARKS/Ref/"

ref_file: "reference.fasta"
trim: 1000
genome_size: 295052
#genome_size: 295179


############ FUSION DES FICHIERS AVANT ENTREE DANS LE PIPELINE #############


### dans le répértoire où se situe l'ensemble des fastq barcodés sorties du séquenceur ###

cat *.fastq > "$i".fastq


##***` Ce Document a pour but d'expliquer le pipeline d'analyse bioinformatique, ainsi que le fonctionnement du SnakeMake utilisé lors de ce projet `***##


# PARTIE CONTROLE DE LA QUALITE #

##`PYCOQC `##

PycoQC : Outil de contrôle qualité, permet de donner une vue d'ensemble de la qualité des bases après le BaseCalling

-Entrée : fichiers summary_sequence.txt produits par le sequenceur (BaseCaller : Guppy)

-Utilité : produit un ensemble de graphiques a partir des fichiers en sortie du basecalling. fournit un ensemble structuré d'analyses des données en entrée a
n de percevoir les éventuels problèmes
de qualité de nos séquences avant la suite des analyses. Parmi les critères de contrôle qualité des données, on peut citer :
    - Basecalled reads length : informations sur la taille des reads totaux.
    -  PHRED score : score de qualité donnant une idée sur le taux d'erreur du BaseCalling, un score de 10 représente 10% de taux d'erreur. 
    - Read quality over experiment time : le score des bases séquencés en fonction de la durée du séquençage
    - Number of reads per barcode : répartition des échantillons du même pool, afin de percevoir d'eventuels problèmes lors de la fabrication des librairies. 


-Sortie : lien HTML vers une visualisation globale de la qualité des données




In [ ]:
### Commande pour executer PycoQC sur nos fichiers ###

/students/BILL/Groupe06/data/F2_salle1/sequencing_summary_FAU70539_204952ac.txt -o pycoQC_output_F2Salle1.html


##` SeqKit  `##

Seqkit Seq : Cette commande permet d’exécuter en partie SeqKit stats produisant des statistiques de base (comme PycoQC)
sur nos bases avec un filtre prenant en compte que les reads au dessus de 500pb.

Seqkit Seq permet également d'executer l'option seqkit Trimm, qui permet d'enlever les séquenceurs adaptatrices utilisées pour le séquençage. Cette étape est nécéssaire pour avoir un bon alignement pour l'étape qui va suivre.

entrée : fichiers Fastq

sortie : fichiers Fastq trimmé (.trimmed)

In [ ]:
 ### Commande pour executer PycoQC sur nos fichiers ###
 
 seqkit stats "$file" >> /students/BILL/Groupe06/sekit/sekit_F2S1 

#   Alignement  #

##`  Minimap2  `##
 
L'outil Minimap2 est un logiciel d'alignement de séquences d'ADN ou d'ARN à haute performance. Il est utilisé pour trouver des régions similaires entre deux séquences. 
L'alignement de ces séquences sur une référence génomique permet de corriger les erreurs d'assemblage et d'améliorer la qualité du génome assemblé.

-Entrée : Un fichier fastq.trimmed  + Un fichier Fasta contenant le génome de référence

-Sortie : des fichiers SAM (Sequence Alignement Mapping) des reads après alignement.

In [ ]:
### Commandes snakeMake pour executer le pipeline ###
## Les lignes de commandes sont celles executées par notre groupe (groupe6) concernant l'échantillon P30.C6 ##

## Première commande sert à donner le chemin du fichier config.yaml necessaire pour obtenir les répértoires d'entrée et de sortie du snakemake, ainsi que le fichier de sortie flagstat pour les statistiques avant alignement ! #
snakemake -s SCRIPTS/Snakefile --cores 1 --keep-going --cluster-config SCRIPTS/config.yaml Analyse/P30/P30C6_all/P30C6_all.trimed1000.flagstat


## Seconde commande Bash permet d'executer la ligne de commande produisant des VCF a partir des  fichiers BAM.mapped.sorted.vcf : donc les fichiers obtenus après execution de minimap2, suivi de SAMTools view (production BAM à partir des SAM), et SAMTools Sort (filtrer les reads mal alignés).

snakemake -s SCRIPTS/Snakefile --cores 1 --keep-going --cluster-config SCRIPTS/config.yaml Analyse/P30/P30C6_all/P30C6_all.trimed1000.mapped.sorted.vcf

# Tri et filtrage des données #

##` SAMTools `##

L'outil SAMtools est un logiciel d'analyse de fichiers de séquençage d'ADN ou d'ARN au format SAM (Sequence Alignment/Map) ou BAM (Binary Alignment/Map). 
Il permet également de lire, écrire, éditer, indexer et visualiser les fichiers de format SAM/BAM/CRAM.
Ce dernier est composé d'un ensemble d'outils permettant le traitement et l'alignement des reads, y compris l'indexation, mais également l'appel de variants (avec l'aide de la suite BCFtools), et une visualisation des alignements.

Entrée  : fichier SAM produit par l'alignement par Minimap2

Sortie : des fichiers BAM (option SAMTools view) depuis les fichiers SAM. Ces derniers vont permettre de produire : des BAM.indexed (SAMTools index), BAM.sorted (SAMTools sort). 

NB : il existe également l'option SAMTools flagstat prend en entrée des fichiers SAM non trié encore, et produit une sortie txt sur le ratio de reads mappés/non_mappés.


#   Variant Calling  #

##`  Sniffles  `##

Sniffles est un logiciel de détection de variants génétiques à partir de données de séquençage de génomes. 
Plus précisément, Sniffles est un outil de variant calling pour la détection des insertions et des délétions de séquences d'ADN à partir de fichiers BAM ou CRAM alignés sur une référence génomique.

Entrée : fichiers BAM triés\filtrés ainsi que les BAM indexés

Sortie : Fichiers VCF (Variant Calling Files)

In [ ]:
### Nous nous pouvions pas avoir accès a la ligne de commande pour  Sniffle executée par snakemake. Cependant dans le gitHub de l'outil en question on peut trouver  la commande suivant pour produire des VCF : ###

sniffles -i mapped_input.bam -v output.vcf


### Ce dernier prend en entrée 1 fichier BAM.mapped.sorted et 1 fichiers BAM.indexed en input, afin de produire un ficheir VCF en output. ###

 #   Visualisation des données #

##`  DeepTools  `##

 DeepTools qui est  est une suite d'outils python  développés pour l'analyse efficace des données de séquençage à haut débit. 
 DeepTools est un ensemble d'outils permettant de modifier des fichiers BAM par exemple avec BAMCoverage comme nous l'avons vu précédemment afin de transformer nos fichiers BAM en BigWig. 
 Avec plotProfile DeepTools permet également de produire des Plots en se basant sur nos données séquencées  pour une meilleure visualisation. 
 DeepTools  réalise également une normalisation des données qui permet la comparaison visuelle de nos séquences avec le génome de référence

 Entrée : fichiers BAM trimmed.mapped.sorted et des fichiers BAM.indexed

 Sortie : fichiers Bedgraph\BigWig (`Option BamCoverage`) ainsi que des fichiers pdf pour la visualisation de la profondeur en mutant (`Option PlotCoverage`)

In [ ]:
### ligne de commande pour executer l'option BamCoverage de DeepTools afin de produire des Bedgraph en output ###

bamCoverage -b P30C6_all.trimed1000.mapped.sorted.bam -o P30C6_all.trimed1000.mapped.sorted.bedgraph --outFileFormat bedgraph --effectiveGenomeSize 295052 --normalizeUsing RPGC


In [ ]:
### Ligne de commande Plot coverage prenant en input les fichiers BAM afin de produire en output un fichier PDF avec des graphiques de la profondeur des reads ###

plotCoverage -b Analyse/P30/P30C6_all/P30C6_all.trimed1000.mapped.sorted.bam -o Analyse/P30/P30C6_all/P30C6_all.trimed1000.mapped.sorted/plotCoverage.pdf --smartlabels --plotFileFormat pdf